In [2]:
%run qsu.ipynb  # color-printing functions
import numpy as np
from qecsim import paulitools as pt
from qecsim.models.generic import DepolarizingErrorModel, BitFlipErrorModel, PhaseFlipErrorModel
from qecsim.models.planar import PlanarCode, PlanarMWPMDecoder, PlanarMPSDecoder
from ti_decoder import ti_calc
from gen_hard_synd import gen_hard_synd

# initialise models
d = 11
my_code = PlanarCode(d, d)
my_error_model = PhaseFlipErrorModel()
my_decoder = PlanarMWPMDecoder()
# print models
print(my_code)
print(my_error_model)
print(my_decoder)

PlanarCode(11, 11)
PhaseFlipErrorModel()
PlanarMWPMDecoder()


In [4]:
# set physical error probability to 10%
error_probability = 0.1

synd_lst, err_lst = gen_hard_synd(d=d, error_probability=error_probability, num_samples=1, seed=1)

error = err_lst[0]
qsu.print_pauli('error:\n{}'.format(my_code.new_pauli(error)))

In [5]:
# syndrome: stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
qsu.print_pauli('syndrome:\n{}'.format(my_code.ascii_art(syndrome)))

Please add the following code to PlanarMPSDecoder class
```python
def calc_coset_probabilities(self, code, syndrome,
            error_model=DepolarizingErrorModel(),  # noqa: B008
            error_probability=0.1, **kwargs):
    """
    See :meth:`qecsim.model.Decoder.decode`

    Note: The optional keyword parameters ``error_model`` and ``error_probability`` are used to determine the prior
    probability distribution for use in the decoding algorithm. Any provided error model must implement
    :meth:`~qecsim.model.ErrorModel.probability_distribution`.

    :param code: Planar code.
    :type code: PlanarCode
    :param syndrome: Syndrome as binary vector.
    :type syndrome: numpy.array (1d)
    :param error_model: Error model. (default=DepolarizingErrorModel())
    :type error_model: ErrorModel
    :param error_probability: Overall probability of an error on a single qubit. (default=0.1)
    :type error_probability: float
    :return: Recovery operation as binary symplectic vector.
    :rtype: numpy.array (1d)
    """
    # any recovery
    any_recovery = self.sample_recovery(code, syndrome)
    # probability distribution
    prob_dist = error_model.probability_distribution(error_probability)
    # coset probabilities, recovery operations
    coset_ps, recoveries = self._coset_probabilities(prob_dist, any_recovery)

    return coset_ps, recoveries
```


In [6]:
# from mps decoder
my_mps_decoder = PlanarMPSDecoder(chi=6)
coset_ps, recoveries = my_mps_decoder.calc_coset_probabilities(
    my_code,
    syndrome,
    error_model=my_error_model,
    error_probability=error_probability)

print('total prob of each coset', coset_ps)
print('Only the first and last are non-zero because only Z errors.')
print('The log of their ratio is:',
      np.log(float(coset_ps[3]) / float(coset_ps[0])))


total prob of each coset (mpf('1.0854150609680467e-28'), mpf('0.0'), mpf('0.0'), mpf('5.5498607152402832e-29'))
Only the first and last are non-zero because only Z errors.
The log of their ratio is: -0.6707747204155996


In [7]:
recovery = recoveries[0].to_bsf()
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(recovery)))

In [8]:
# couriosity:recovery ^ error
qsu.print_pauli('recovery ^ error:\n{}'.format(my_code.new_pauli(recovery ^ error)))

In [6]:
ti_calc(recovery,
           error_probability=error_probability,
           d=d,
           num_int_steps=201)


l = 0.00, energy derivative = -19.6440
l = 0.01, energy derivative = -20.1280
l = 0.01, energy derivative = -20.3560
l = 0.01, energy derivative = -20.2440
l = 0.02, energy derivative = -19.8440
l = 0.03, energy derivative = -20.0720
l = 0.03, energy derivative = -20.9160
l = 0.04, energy derivative = -19.6280
l = 0.04, energy derivative = -19.9840
l = 0.04, energy derivative = -19.9960
l = 0.05, energy derivative = -19.8640
l = 0.06, energy derivative = -19.9640
l = 0.06, energy derivative = -19.8040
l = 0.07, energy derivative = -19.1080
l = 0.07, energy derivative = -18.6960
l = 0.07, energy derivative = -19.3960
l = 0.08, energy derivative = -18.8840
l = 0.09, energy derivative = -18.9360
l = 0.09, energy derivative = -18.8720
l = 0.10, energy derivative = -19.2800
l = 0.10, energy derivative = -19.0360
l = 0.10, energy derivative = -18.3880
l = 0.11, energy derivative = -18.4080
l = 0.12, energy derivative = -18.7760
l = 0.12, energy derivative = -18.9880
l = 0.12, energy derivati

-0.45173188275543336

## Try starting configuration from MWPM

In [7]:
mwpm = PlanarMWPMDecoder()
mwpm_recovery = mwpm.decode(my_code, syndrome)
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(mwpm_recovery)))

In [8]:
ti_calc(mwpm_recovery,
           error_probability=error_probability,
           d=d,
           num_int_steps=201)

l = 0.00, energy derivative = -19.9720
l = 0.01, energy derivative = -20.1600
l = 0.01, energy derivative = -20.3000
l = 0.01, energy derivative = -20.3080
l = 0.02, energy derivative = -20.1280
l = 0.03, energy derivative = -20.2200
l = 0.03, energy derivative = -20.6240
l = 0.04, energy derivative = -19.8000
l = 0.04, energy derivative = -20.1440
l = 0.04, energy derivative = -20.3080
l = 0.05, energy derivative = -20.1200
l = 0.06, energy derivative = -19.9440
l = 0.06, energy derivative = -20.4280
l = 0.07, energy derivative = -19.8640
l = 0.07, energy derivative = -19.5560
l = 0.07, energy derivative = -18.9360
l = 0.08, energy derivative = -19.6600
l = 0.09, energy derivative = -19.3880
l = 0.09, energy derivative = -19.2880
l = 0.10, energy derivative = -19.1840
l = 0.10, energy derivative = -19.2360
l = 0.10, energy derivative = -18.6960
l = 0.11, energy derivative = -19.1640
l = 0.12, energy derivative = -19.5320
l = 0.12, energy derivative = -19.6560
l = 0.12, energy derivati

0.05494154589896081